## Numbers Dataset

We have a dataset of handwritten digits from 0 to 9

We need some library to read the images, I've chosen the pillow library which comes with Anaconda, if you need to 

pip install Pillow

should install it for you

In [33]:
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
number_of_classes = 10

##Printing Examples
def printexamples(X):
    examples_per_class = 10
    pos = 0
    for cls in range(number_of_classes):
        idxs = range(pos,pos+500)
        idxs = np.random.choice(idxs, examples_per_class, replace=False)
        for i, idx in enumerate(idxs):
            plt.subplot(examples_per_class, number_of_classes, i * number_of_classes + cls + 1)
            plt.imshow(X[idx].reshape(20,20).astype('uint8'), cmap='gray')
            plt.axis('off')
            if i == 0:
                plt.title(str(cls))
        pos = pos+500
    plt.show()

In [ ]:
# get the list of all the files in digits.zip/data and create filelist
filelist = glob.glob('data/*/*.txt')

#create a file list
X = np.array([np.array(Image.open(f).convert('L').resize((64, 64))).flatten() for f in filelist])

# Step 3: Extract class labels from filenames
# Assuming filenames are like 'category_0_image1.jpg', 'category_1_image2.jpg', etc.
y = np.array([int(os.path.basename(f).split('_')[1]) for f in filelist])

# Step 4: Ensure X and y are properly structured
print(X.shape)  # Should be (num_images, flattened_image_size)
print(y.shape)  # Should be (num_images,)
print(y)  # Check labels

a = [s.split() for s in " ".join(filelist).split("\\") if s]
#a = np.array(filename.split('\\')[1] for filename in filelist, dtype=int)
#print(a.shape)


(0,)
(0,)
[]
